<a href="https://colab.research.google.com/github/yubraaj11/Adobe-XD-Projects/blob/main/Transfer_Learning_ImageNet(Gritfeat_data).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

In [ ]:
dataset_dir = "/content/drive/MyDrive/images"

In [ ]:
num_classes = len(os.listdir(dataset_dir))
batch_size = 32
num_epochs = 10

In [ ]:
num_classes

5

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [ ]:
dataset = datasets.ImageFolder(dataset_dir, transform=data_transforms['train'])

In [ ]:
train_indices, test_indices = train_test_split(list(range(len(dataset))),
                                               test_size=0.2,
                                               random_state=42,
                                               stratify=dataset.targets)

In [ ]:
train_dataset = torch.utils.data.Subset(dataset, train_indices)
test_dataset = torch.utils.data.Subset(dataset, test_indices)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [ ]:
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
for epoch in range(num_epochs):
    # Training phase
    model.train()
    running_loss = 0.0
    running_corrects = 0
    total_samples = 0

    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        total_samples += labels.size(0)

    epoch_train_loss = running_loss / total_samples
    epoch_train_acc = running_corrects.double() / total_samples

    print('Epoch [{}/{}], Train Loss: {:.4f}, Train Acc: {:.4f}'
          .format(epoch+1, num_epochs, epoch_train_loss, epoch_train_acc))

    # Validation phase
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    total_samples = 0

    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        total_samples += labels.size(0)

    epoch_val_loss = running_loss / total_samples
    epoch_val_acc = running_corrects.double() / total_samples

    print('Epoch [{}/{}], Validation Loss: {:.4f}, Validation Acc: {:.4f}'
          .format(epoch+1, num_epochs, epoch_val_loss, epoch_val_acc))


Epoch [1/10], Train Loss: 0.9995, Train Acc: 0.6270
Epoch [1/10], Validation Loss: 0.7508, Validation Acc: 0.7480
Epoch [2/10], Train Loss: 0.7383, Train Acc: 0.7550
Epoch [2/10], Validation Loss: 0.7050, Validation Acc: 0.7700
Epoch [3/10], Train Loss: 0.6673, Train Acc: 0.7685
Epoch [3/10], Validation Loss: 0.6271, Validation Acc: 0.7860
Epoch [4/10], Train Loss: 0.6442, Train Acc: 0.7770
Epoch [4/10], Validation Loss: 0.6529, Validation Acc: 0.7560
Epoch [5/10], Train Loss: 0.6174, Train Acc: 0.7935
Epoch [5/10], Validation Loss: 0.5232, Validation Acc: 0.8140
Epoch [6/10], Train Loss: 0.6124, Train Acc: 0.7885
Epoch [6/10], Validation Loss: 0.5355, Validation Acc: 0.7940
Epoch [7/10], Train Loss: 0.5614, Train Acc: 0.8055
Epoch [7/10], Validation Loss: 0.5406, Validation Acc: 0.8200
Epoch [8/10], Train Loss: 0.5615, Train Acc: 0.8055
Epoch [8/10], Validation Loss: 0.4980, Validation Acc: 0.8220
Epoch [9/10], Train Loss: 0.5660, Train Acc: 0.8005
Epoch [9/10], Validation Loss: 0.507

In [22]:
torch.save(model.state_dict(), 'model_weights.pth')

In [28]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image

# Define class labels (replace with your actual class labels)
class_labels = ['class 0', 'class 2', 'class 4', 'class 6', 'class 9']

# Load the trained model
model = models.resnet18(pretrained=False)  # Assuming you have already trained and saved the model
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_labels))  # Adjusting the number of output classes

# Load the trained model weights
model.load_state_dict(torch.load('model_weights.pth', map_location=torch.device('cpu')))  # Load saved model weights

# Set the model to evaluation mode
model.eval()

# Define image transformation
data_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load and preprocess the new image
image_path = '/content/drive/MyDrive/images/2/036ed50c-b647-40a9-bbbd-92b2037e4660.TIF'
image = Image.open(image_path)
image = image.convert("RGB")
image_tensor = data_transform(image).unsqueeze(0)  # Add batch dimension

# Perform inference
with torch.no_grad():
    outputs = model(image_tensor)
    _, predicted = torch.max(outputs, 1)
    predicted_class = class_labels[predicted.item()]

print(f"The predicted class for the new image is: {predicted_class}")


The predicted class for the new image is: class 4
